**GET DATA**

Summary of this notebook: ...

Todo:

1. Self supervised representation learning using rotation as a pretext task
2. Taking in an image I , rotating it by an a set angle A--> rotated image (I_A), rotated angle A (actual label) (Done)
3. Dataloader that does this somehow (Done)
4. Some model-> ResNet50  + new full-connected layer (feature vector-> probability that image belongs to one of n classes) (Done-ish)
5. Additional training with the above model minimizing the loss wrt. actual rotated angle
f(I_A)=min[L(A_predict,A)]
6. Save this new model

7. Proof of concept:
Imagenet-> Train on Caltech validation on caltech




Definition of Done: ...

# Imports


In [ ]:
# Mount Google drive
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Change to current dataset
os.chdir("/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code")

In [ ]:
# !pip install split-folders
# dataset = "UCMerced_LandUse"
# dataPath = "Datasets/"+ dataset+ "/Images"
# import split_folders
# split_folders.ratio(dataPath+'/',output=dataPath+'/Splits_2/', seed=1337, ratio=(.8, .2, .0)) # default values


In [ ]:
# Imports from Colab 2
import os
import math
import numpy as np
import pickle
import keras
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import pretrained model
from tensorflow.keras.applications import MobileNet, ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [ ]:
# Imports for Colab 6
import cv2 # Read raw image
import glob
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from scipy import ndimage # For rotation task or
import imutils
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator

In [ ]:
# Check to see if GPU is being used
tensorflow.test.gpu_device_name()

'/device:GPU:0'

# Data Augmentation/Analysis

In [ ]:
# Raw dataset/path/model paths
dataset = "UCMerced_LandUse"
dataPath = ("Datasets/"+ dataset+ "/Splits_2")
modelName = "MobileNet"

In [ ]:
#  Image modification functions- to plot image as well as to do a rotation task
def disp_mpl(im_arr): # Matplotlib imshow
  plt.imshow(im_arr)
  plt.show()

def disp_cv2(im_arr): # OpenCV2 image
  cv2_imshow(im_arr)

def basic_rotate(im_arr,angle): # Basic Scipy ndimage rotation. Results in a black border
  return ndimage.rotate(im_arr,angle,reshape=True)

# If we really want to do this: https://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result


def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

In [ ]:
# Test rotation
# angle=45
# sample_image_path=dataPath+'/river/river00.tif'
# image=cv2.imread(sample_image_path)
# h,w,_=image.shape
# rotated_image=rotate_image(image,angle)

# image_rotated_cropped = crop_around_center(rotated_image,*largest_rotated_rect(w,h,math.radians(angle)))


In [ ]:
# disp_cv2(image)

In [ ]:
# image.shape

In [ ]:
# disp_cv2(rotated_image)

In [ ]:
# disp_cv2(image_rotated_cropped)

In [ ]:
# image_rotated_cropped.shape

# Dataloader creation and test

In [ ]:
# Glob through all the images we have in the current directory
# image_paths=glob.glob(dataPath+'/**/*.tif')



In [ ]:
train_dataGenerator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   channel_shift_range=0.1)

valid_dataGenerator=ImageDataGenerator(preprocessing_function=preprocess_input)


batch_size = 32
trainGenerator = train_dataGenerator.flow_from_directory(
        dataPath+'/train/',
        target_size=(224, 224),
        batch_size= batch_size,
        class_mode= None, 
        shuffle = False)

testGenerator=valid_dataGenerator.flow_from_directory(
        dataPath+'/val/',
        target_size=(224, 224),
        batch_size= batch_size,
        class_mode= None, 
        shuffle = False)

Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


In [ ]:
# X here is the yield output from a data generator (test)
# def batch_augment(flow_gen,n_classes):
#   print(flow_gen.shape)
#   for batch_images in flow_gen:
#     X=np.empty_like(flow_gen.shape)
#     y=np.empty((len(flow_gen),n_classes))

#     for im in batch_images:
#       print(im.shape)
#       h,w,_=im.shape
#       angle_categorical=np.random.randint(low=0,high=n_classes)
#       angle=angle_categorical*90
#       rotated_image=rotate_image(im,angle)
      
#       image_rotated_cropped = crop_around_center(rotated_image,*largest_rotated_rect(w,h,math.radians(angle)))
#       h,w,_=image_rotated_cropped.shape
      
#       center=(h//2,w//2)
#       d_h=246
#       X[i,]= image_rotated_cropped[int(center[0]-d_h/2):int(center[0]+d_h/2), int(center[1]-d_h/2):int(center[1]+d_h/2)]
#       y[i]=angle_categorical
#     yield X,keras.utils.to_categorical(y, num_classes=n_classes)

    # X here is the yield output from a data generator (test)
def batch_augment(flow_gen,dim,n_classes=4):
  
  while True:
    X=np.empty((0,*dim))
    y=np.empty((0,1))
    for i,batch_im in enumerate(flow_gen[0]):
      h,w,_=batch_im.shape
      angle_categorical=np.random.randint(low=0,high=n_classes)
      angle=angle_categorical*90
      rotated_image=rotate_image(batch_im,angle)
        
      image_rotated_cropped = crop_around_center(rotated_image,*largest_rotated_rect(w,h,math.radians(angle)))
      h_c,w_c,_=image_rotated_cropped.shape
      # print(h_c,w_c)
      center=(h_c//2,w_c//2)
      # print(center)
      
      d_h=200
      # print(int(center[0]-d_h/2),int(center[0]+d_h/2))
      # print(int(center[1]-d_h/2),int(center[1]+d_h/2))
      aug_batch_im=image_rotated_cropped[int(center[0]-d_h/2):int(center[0]+d_h/2), int(center[1]-d_h/2):int(center[1]+d_h/2)]
      aug_batch_label=angle_categorical
      # print(X.shape,np.array(aug_batch_im).shape)
      # print(y.shape,np.array([aug_batch_label]).shape)
      X=np.append(X,np.expand_dims(aug_batch_im, axis=0),axis=0)
      y=np.append(y,np.expand_dims(np.array([aug_batch_label]), axis=0),axis=0)
    yield X,keras.utils.to_categorical(y, num_classes=n_classes)

In [ ]:
# augmented_generator=batch_augment(trainGenerator,(200,200,3),4)

In [ ]:
# augmented_testgenerator=batch_augment(testGenerator,(200,200,3),4)

In [ ]:
# Link: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

# Create a custom dataloader that performs a rotation task as well as generates a pseudo-label (in __data_generation)
# Put in the output image size and number of classes as a variable


# class RandomFramesFromPathsToVideos(data_utils.Sequence):
'''
class DataGenerator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, dim=(246,246,3),
                 n_classes=4, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)
        for i in range(len(list_IDs_temp)): 
          temp_image=cv2.imread(list_IDs_temp[i])
          h,w,_=temp_image.shape
          angle_categorical=np.random.randint(low=0,high=4) # 0,1,2,3
          angle=angle_categorical*90
          rotated_image=rotate_image(temp_image,angle)

          image_rotated_cropped = crop_around_center(
            rotated_image,
            *largest_rotated_rect(
                w,
                h,
                math.radians(angle)
            )
          )
          h,w,_=image_rotated_cropped.shape
          center=(h//2,w//2)
          d_h=246
          X[i,]= image_rotated_cropped[int(center[0]-d_h/2):int(center[0]+d_h/2), int(center[1]-d_h/2):int(center[1]+d_h/2)]
          y[i]=angle_categorical

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

'''


In [ ]:
# Instantiate train generator
# train_generator=DataGenerator(list_IDs=image_paths)

In [ ]:
# Grab batch at index 0
# x,y=train_generator.__getitem__(0)

In [ ]:
# Test shapes
# print(x.shape)
# print(y.shape)

In [ ]:
# Validate the data augmentation
# disp_cv2(x[2])
# print(y[2])

# Model creation

In [ ]:
# Base model is a Mobile net (from Colab 2)
base_model = ResNet50(
    input_shape = (200, 200, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = "max"
)
print(base_model.output.shape)

# Head model consists of a 1 Dense layer (could be improved later) to output for the number of classes
head_model=Flatten()(base_model.output)
head_model=Dense(512,activation='relu')(head_model)
head_model=Dropout(0.3)(head_model)
head_model=Dense(128,activation='relu')(head_model)
head_model=Dropout(0.2)(head_model)
predictions=Dense(units=8,activation='softmax')(head_model)


(None, 2048)


In [ ]:
# Base model is a Mobile net (from Colab 2)
# base_model_untr = MobileNet(
#     input_shape = (200, 200, 3),
#     include_top = False,
#     weights = None,
#     pooling = "max"
# )
# print(base_model_untr.output.shape)

# # Head model consists of a 1 Dense layer (could be improved later) to output for the number of classes
# head_model_untr=Flatten()(base_model_untr.output)
# head_model_untr=Dense(256,activation='relu')(head_model_untr)
# head_model_untr=Dropout(0.2)(head_model_untr)
# predictions_untr=Dense(units=4,activation='softmax')(head_model_untr)

In [ ]:
# complete_model_untr=Model(base_model_untr.input,predictions_untr)

In [ ]:
# Stitch the models together (not compiled yet)
complete_model=Model(base_model.input,predictions)
# complete_model.summary()

# Model Training

complete_model=[base_model,head_model]
Base Model--> Convolution layers trained on Imagenet
Head Model--> Dense layer(s) w/ no training

1. Train dense layer to be good
2. Train part of the base model to be a bit better.

In [ ]:
opt=Adam(learning_rate=1e-3)

## Training Cycles
#### First: Freeze the base model and just train the dense models at the head

In [ ]:
for l in base_model.layers:
  l.trainable=False

In [ ]:
complete_model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
complete_model.fit(batch_augment(trainGenerator,(200,200,3),4),
                   validation_data=batch_augment(testGenerator,(200,200,3),4),
                   steps_per_epoch=1680/32,
                   validation_steps=420/32,
                   epochs=5)


ValueError: ignored

#### Second: Unfreeze the last "part" of the base model and just train

In [ ]:
trainGenerator.reset()
testGenerator.reset()

In [ ]:
start_from=int(len(base_model.layers)*(1/2))

In [ ]:
for layer in base_model.layers[start_from:]:
  layer.trainable=True

In [ ]:
complete_model.fit(batch_augment(trainGenerator,(200,200,3),4),
                   validation_data=batch_augment(testGenerator,(200,200,3),4),
                   steps_per_epoch=1680/32,
                   validation_steps=420/32,
                   epochs=10)



Epoch 1/10
53/52 [==============================] - 32s 607ms/step - loss: 0.8695 - accuracy: 0.5419 - val_loss: 2.0356 - val_accuracy: 0.3147
Epoch 2/10
53/52 [==============================] - 32s 610ms/step - loss: 0.8161 - accuracy: 0.5772 - val_loss: 1.7743 - val_accuracy: 0.3750
Epoch 3/10
53/52 [==============================] - 32s 608ms/step - loss: 0.7829 - accuracy: 0.5820 - val_loss: 2.3329 - val_accuracy: 0.2790
Epoch 4/10
53/52 [==============================] - 33s 616ms/step - loss: 0.7112 - accuracy: 0.6291 - val_loss: 2.1676 - val_accuracy: 0.3214
Epoch 5/10
53/52 [==============================] - 32s 605ms/step - loss: 0.7230 - accuracy: 0.6073 - val_loss: 1.6936 - val_accuracy: 0.3304
Epoch 6/10
53/52 [==============================] - 32s 611ms/step - loss: 0.6953 - accuracy: 0.6327 - val_loss: 1.9338 - val_accuracy: 0.2991
Epoch 7/10
53/52 [==============================] - 32s 602ms/step - loss: 0.6413 - accuracy: 0.6781 - val_loss: 2.1702 - val_accuracy: 0.3170

In [ ]:
complete_model.save('Models/Rotation_Resnet50')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Models/Rotation_Resnet50/assets


## Test on untrained MobileNet+Dense layer


In [ ]:
# for l in base_model_untr.layers:
#   l.trainable=False

In [ ]:
# complete_model_untr.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
# complete_model_untr.fit(batch_augment(trainGenerator,(200,200,3),4),steps_per_epoch=2100/32,epochs=5)

In [ ]:
# trainGenerator.reset()
# for layer in base_model_untr.layers[start_from:]:
#   layer.trainable=True
# complete_model_untr.fit(batch_augment(trainGenerator,(200,200,3),4),steps_per_epoch=2100/32,epochs=5)
